# **Select background subtracted images**


In [ ]:
# Import packages
import sys
import numpy as np
from pathlib import Path

src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
    
from src.d01_init_proc.subtractbg import batch_subtract_bg

%matplotlib notebook
%matplotlib inline

import pandas as pd


## Import images

##### **Add path for a folder of images to be analyzed.**

In [ ]:
subtractbg_params_path = Path(input('Please enter the full path for the subtractbg_params.csv file:\n'))

In [ ]:
def process_bgsub_files(subtractbg_params_path):
    
    df = pd.read_csv(subtractbg_params_path)
    df['bgsub_basename'] = [(imgname.split('.')[0] + '_' + bgsub_time) for imgname, bgsub_time in zip(df['Image name'], df['Bgsub_time'])]
    selected_bgsub_ometifs = list(df['bgsub_basename'].loc[(df['Keep? Y/N']=='Y')])

    bgsub_dirpath = subtractbg_params_path.parent
    bgsub_selected_dirpath = bgsub_dirpath.parent / 'bg_subtracted_selected'
    bgsub_selected_figs_dirpath = bgsub_selected_dirpath / 'bg_sb_figs'
    bgsub_selected_figs_dirpath.mkdir(parents=True, exist_ok=True)

    num_moved = 0

    for bgsub_ometif in selected_bgsub_ometifs:
        orig_path = bgsub_dirpath / (bgsub_ometif + '.ome.tif')
        if (orig_path.is_file()):
            num_moved = num_moved + 1
            orig_path.rename(bgsub_selected_dirpath / (bgsub_ometif + '.ome.tif'))
        orig_fig_path = bgsub_dirpath / 'bg_sb_figs' / (bgsub_ometif + '.png')
        if (orig_fig_path.is_file()):
            orig_fig_path.rename(bgsub_selected_figs_dirpath / (bgsub_ometif + '.png'))
        
    print(f'{num_moved} files moved')
    
    redo_bgsub_idc = list(df['Index'].loc[(df['Redo? Y/N']=='Y')])
    return redo_bgsub_idc
    
redo_bgsub_idc = process_bgsub_files(subtractbg_params_path)
print(redo_bgsub_idc)

In [ ]:
input_dirpath = input()

In [ ]:
params_to_test = []

In [ ]:
ch_to_process=(1, 2)

params = {}
# 100 percentile is equivalent to no clipping at all
params['outlier_percentiles'] = [93, 70]

# 0 sigma_smoothing = no smoothing
params['sigmas_smoothing'] = [0, 0.5]
params_to_test.append(params)

for params in params_to_test:
    print(params)

In [ ]:
subset=redo_bgsub_idc
batch_subtract_bg(input_dirpath, params_to_test, ch_to_process=ch_to_process, subset=subset, target_perc_grayval=50)